In [2]:
def calc_solution_value(l, L, N, M):# 1 * w[0] + 2 * w[1] + .... + M * w[M-1]

    w = deepcopy(L)
    for i in range(N):
        for j in range(M):
            if l[i] <= w[j]:
                w[j] -= l[i]
                break
    
    w.sort(reverse = True)
    num_stock = [i for i in range(1, M + 1)]
    result = 0
    for i in range(M):
        result += w[i] * num_stock[i] #skalarni proizvod?
    return result

In [6]:
import random

In [ ]:
def make_small_change(l, N):
    idxs = [i for i in range(N)]
    random_length = random.randrange(1, N)
    chosen = random.sample(idxs, k = random_length)
    for i in range(random_length):
        for j in range(chosen[i] + 1, N):
            if l[chosen[i]] != l[j]:
                tmp = l[chosen[i]]
                l[chosen[i]] = l[j]
                l[j] = tmp
                break
    

In [4]:
from copy import deepcopy

In [5]:
def initialize(l): # l je lista porudzbina tkd ako dobijem 2 od ove velicine i 3 od ove i blabla
    #onda treba sve to u jedan niz
    l.sort(reverse = True) #nisam sig bas dal l da sortiram


In [ ]:
# nk - The number of the ordered small items by the length of l k .
# N - The number of all small items.
# li - The length of the i th ordered item.
# M - The number of all large objects. - moze da ne bude dato u zadatku vec da se izracuna u initialize f-ji
# Lj - The length of the j th large object.
# wj - The trim loss appeared on j th large object.
# Xij - A binary variable which is equal to one if the i th Item is cut from the j th large object and
# zero otherwise.
# wj = Lj − for i in range(N) for j in range(M)(li * xij) - calc trim loss 
# 0 <= wj <= Lj - logicno

# 0 < j < M - materijal koji imamo
# 0 < i < N - broj porucenih 

In [ ]:
import math

In [1]:
def simulated_annealing(l, L, N, M):
    L.sort(reverse = True)
    solution = deepcopy(l)
    initialize(solution)
    value = calc_solution_value(solution, L, N, M)
    best_solution = deepcopy(solution)
    best_value = value
    T = 1000
    a = 0.9
    min_temperature = 1e-3  

    
    while T > 1e-3:
        new_solution = deepcopy(solution)
        make_small_change(new_solution, N)
        new_value = calc_solution_value(new_solution, L, N, M)
        
        diff = new_value - value
        if diff < 0:
            solution = deepcopy(new_solution)
            value = new_value
            if new_value < best_value:
                best_value = new_value
                best_solution = deepcopy(new_solution)
        else:
            if (math.e)**((-diff) / T)
                value = new_value
                solution = deepcopy(new_solution)
        T = a * T
        
    return best_solution, best_value 

In [2]:
def calc_M(l, N):
    stock_len = 100
    stocks = [stock_len]
    num_stock = 1
    for i in range(N):
        for j in range(num_stock):
            if stocks[j] >= l[i]:
                stocks[num_stock] -= l[i]
            else:
                if j == (num_stock - 1):
                    num_stock += 1
                    stocks.append(stock_len)
    return num_stock 

In [ ]:
N = len(l)
M = calc_M(l, N)

In [ ]:
solution, value = simulated_annealing(l, L, N, M)
print(solution, value)